# BM3D Inference on EM data

# Gather

In [1]:
import sys
# sys.path.append("../../")

In [15]:
from PIL import Image
import numpy as np
from skimage import filters
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.measure import compare_ssim, compare_psnr
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from path import Path
from collections import defaultdict

In [3]:
import bm3d

In [4]:
bilin_pth = Path('/Users/alaa/data/LR-Bilinear/real-world_SEM/')
target_pth = Path('/Users/alaa/data/HR/real-world_SEM/')
output_pth = Path('/Users/alaa/data/LR-BM3D/real-world_SEM/')

In [5]:
bilin_files = sorted(list(bilin_pth.glob('*.tif')))
target_files = sorted(list(target_pth.glob('*.tif')))


len(bilin_files), len(target_files)

(42, 42, 42)

# Evaluate

In [6]:
def normalize_arr(arr):
    return (arr - np.min(arr))/np.ptp(arr)

In [16]:
def do_inference(sigma=30):
    bm3d_scores = defaultdict(list)
    print(f'Computing Inferences using BM3D params sigma_psd: {sigma}')
    with tqdm(total=42) as pbar:
        for bilin_pth, target_pth in zip(bilin_files, target_files):
            bilin_img = Image.open(bilin_pth)
            bilin_arr = np.asarray(bilin_img, np.uint8)

            target_img = Image.open(target_pth)
            target_arr = np.asarray(target_img, np.uint8)

            norm_arr = normalize_arr(bilin_arr)
            bm3d_arr = bm3d.bm3d(norm_arr, sigma_psd=sigma/255, 
                                 stage_arg=bm3d.BM3DStages.ALL_STAGES)
            bm3d_arr = np.asarray(bm3d_arr*bilin_arr.max(), np.uint8)
            bm3d_psnr = peak_signal_noise_ratio(target_arr, bm3d_arr, data_range=255)
            bm3d_ssim = structural_similarity(target_arr, bm3d_arr, data_range=255)
            bm3d_scores['psnr'].append(bm3d_psnr)
            bm3d_scores['ssim'].append(bm3d_ssim)
            bm3d_img = Image.fromarray(bm3d_arr)
            save_pth = str(output_pth/bilin_pth.name).replace('bilinear', 'bm3d')
            pbar.update(1)

    bm3d_psnr = np.mean(bm3d_scores['psnr'])
    bm3d_ssim = np.mean(bm3d_scores['ssim'])
    print(f'\t BM3D: PSNR = {bm3d_psnr} SSIM = {bm3d_ssim}\n')

    
#     print(f'Saved BM3D Inference in {save_pth}')

# Hyperparameter Tune

In [17]:
all_sigmas = [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
for sigma in all_sigmas:
    do_inference(sigma=sigma)

  0%|          | 0/42 [00:00<?, ?it/s]

Computing Inferences using BM3D params sigma_psd: 1


  0%|          | 0/42 [00:00<?, ?it/s]

	 BM3D: PSNR = 24.537993013076207 SSIM = 0.4998061701997991

Computing Inferences using BM3D params sigma_psd: 5


  0%|          | 0/42 [00:00<?, ?it/s]

...

	 BM3D: PSNR = 25.339135539147648 SSIM = 0.5564856298104969

Computing Inferences using BM3D params sigma_psd: 90


  0%|          | 0/42 [00:00<?, ?it/s]

	 BM3D: PSNR = 25.334374335718074 SSIM = 0.5553119925213826

Computing Inferences using BM3D params sigma_psd: 100


100%|██████████| 42/42 [04:08<00:00,  5.91s/it]

	 BM3D: PSNR = 25.32638373005232 SSIM = 0.5542720726513929



# Hyperparameter Fine-Tune

In [18]:
all_sigmas = [62, 65, 68, 73, 77, 82, 87]
for sigma in all_sigmas:
    do_inference(sigma=sigma)

  0%|          | 0/42 [00:00<?, ?it/s]

Computing Inferences using BM3D params sigma_psd: 62


  0%|          | 0/42 [00:00<?, ?it/s]

	 BM3D: PSNR = 25.326832069405867 SSIM = 0.5582725039202993

Computing Inferences using BM3D params sigma_psd: 65


  0%|          | 0/42 [00:00<?, ?it/s]

...

	 BM3D: PSNR = 25.339551774035687 SSIM = 0.5568377197085793

Computing Inferences using BM3D params sigma_psd: 82


  0%|          | 0/42 [00:00<?, ?it/s]

	 BM3D: PSNR = 25.338379733969937 SSIM = 0.5562399103150357

Computing Inferences using BM3D params sigma_psd: 87


100%|██████████| 42/42 [04:07<00:00,  5.90s/it]

	 BM3D: PSNR = 25.33645729660039 SSIM = 0.555656528362417



In [19]:
all_sigmas = [55, 57, 59]
for sigma in all_sigmas:
    do_inference(sigma=sigma)

  0%|          | 0/42 [00:00<?, ?it/s]

Computing Inferences using BM3D params sigma_psd: 55


  0%|          | 0/42 [00:00<?, ?it/s]

	 BM3D: PSNR = 25.310549749687986 SSIM = 0.5585307355816501

Computing Inferences using BM3D params sigma_psd: 57


  0%|          | 0/42 [00:00<?, ?it/s]

	 BM3D: PSNR = 25.31604901197451 SSIM = 0.5584942699608583

Computing Inferences using BM3D params sigma_psd: 59


100%|██████████| 42/42 [04:11<00:00,  5.98s/it]

	 BM3D: PSNR = 25.320780045702367 SSIM = 0.5584344589997331

